In [143]:
import os 
import caer
import canaro
import cv2 as cv
import numpy as np
import gc

In [144]:
IMG_SIZE = (144,144)
channels = 1
char_path = '/Users/hetan2/Desktop/simpsons /archive/simpsons_dataset'

In [145]:
char_dict ={}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
char_dict = caer.sort_dict(char_dict, descending= True)
char_dict

[('homer_simpson', 2246),
 ('ned_flanders', 1454),
 ('moe_szyslak', 1452),
 ('lisa_simpson', 1354),
 ('bart_simpson', 1342),
 ('marge_simpson', 1291),
 ('krusty_the_clown', 1206),
 ('principal_skinner', 1194),
 ('charles_montgomery_burns', 1193),
 ('milhouse_van_houten', 1079),
 ('chief_wiggum', 986),
 ('abraham_grampa_simpson', 913),
 ('sideshow_bob', 877),
 ('apu_nahasapeemapetilon', 623),
 ('kent_brockman', 498),
 ('comic_book_guy', 469),
 ('edna_krabappel', 457),
 ('nelson_muntz', 358),
 ('lenny_leonard', 310),
 ('mayor_quimby', 246),
 ('waylon_smithers', 181),
 ('maggie_simpson', 128),
 ('groundskeeper_willie', 121),
 ('barney_gumble', 106),
 ('selma_bouvier', 103),
 ('carl_carlson', 98),
 ('ralph_wiggum', 89),
 ('patty_bouvier', 72),
 ('martin_prince', 71),
 ('professor_john_frink', 65),
 ('snake_jailbird', 55),
 ('cletus_spuckler', 47),
 ('rainier_wolfcastle', 45),
 ('simpsons_dataset', 42),
 ('agnes_skinner', 42),
 ('sideshow_mel', 40),
 ('otto_mann', 32),
 ('fat_tony', 27),
 (

In [146]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count+=1
    if count>=10:
        break
characters

['homer_simpson',
 'ned_flanders',
 'moe_szyslak',
 'lisa_simpson',
 'bart_simpson',
 'marge_simpson',
 'krusty_the_clown',
 'principal_skinner',
 'charles_montgomery_burns',
 'milhouse_van_houten']

In [147]:
# create training data
train = caer.preprocess_from_dir(char_path, characters, channels= channels, IMG_SIZE=IMG_SIZE, isShuffle= True)

[INFO] Could not find a file to load from. Generating the training data
----------------------------------------------
[INFO] At 1000 files
[INFO] At 2000 files
[INFO] 2246 files found in 0.0033631324768066406s
[INFO] At 1000 files
[INFO] 1454 files found in 0.001461029052734375s
[INFO] At 1000 files
[INFO] 1452 files found in 0.001447916030883789s
[INFO] At 1000 files
[INFO] 1354 files found in 0.0013468265533447266s
[INFO] At 1000 files
[INFO] 1342 files found in 0.0013051033020019531s
[INFO] At 1000 files
[INFO] 1291 files found in 0.0013167858123779297s
[INFO] At 1000 files
[INFO] 1206 files found in 0.0011951923370361328s
[INFO] At 1000 files
[INFO] 1194 files found in 0.0015878677368164062s
[INFO] At 1000 files
[INFO] 1193 files found in 0.0012159347534179688s
[INFO] At 1000 files
[INFO] 1079 files found in 0.0010669231414794922s
----------------------------------------------
[INFO] 13811 files preprocessed! Took 0m 12s


In [148]:
len(train)

13811

In [149]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [150]:
from tensorflow.keras.utils import to_categorical
#normalize the featureSet (0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [151]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)


In [152]:
del train
del featureSet
del labels
gc.collect()

0

In [153]:
BATCH_SIZE = 32
EPOCHS = 5

In [154]:
# image data generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images up to 20 degrees
    width_shift_range=0.2,  # Shift images horizontally up to 20% of width
    height_shift_range=0.2,  # Shift images vertically up to 20% of height
    horizontal_flip=True,  # Flip images horizontally with 50% probability
)

train_gen = datagen.flow(x_train, y_train, batch_size =BATCH_SIZE)

In [155]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [156]:
from tensorflow.keras.models import Sequential
models = Sequential()


In [157]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(144, 144, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout with 25% probability

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout with 25% probability

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Add dropout with 25% probability

# Flatten the output to feed into dense layers
model.add(Flatten())

# Add dense layers for classification (adjust number of units based on your data)
model.add(Dense(256, activation='relu'))  # Example dense layer
model.add(Dropout(0.5))  # Increase dropout for deeper models
model.add(Dense(len(characters), activation='softmax'))  # Final output layer

# Compile the model (adjust optimizer and loss function as needed)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [158]:
learning_rate = 0.005
momentum = 0.9
nesterov = True
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [159]:
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 142, 142, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 71, 71, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 71, 71, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 69, 69, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 34, 34, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 34, 34, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,484,106 (32.36 MB)

 Trainable params: 8,484,106 (32.36 MB)

 Non-trainable params: 0 (0.00 B)

In [160]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [161]:
print(model.input_shape)

(None, 144, 144, 1)


In [162]:
training = model.fit(train_gen,steps_per_epoch= len(x_train)//BATCH_SIZE,
                      epochs= EPOCHS, 
                      validation_data=(x_val, y_val),
                     validation_steps = len(y_val)//BATCH_SIZE,
                     callbacks = callbacks_list)

Epoch 1/5
345/345 ━━━━━━━━━━━━━━━━━━━━ 76s 218ms/step - accuracy: 0.1193 - loss: 0.3654 - val_accuracy: 0.1664 - val_loss: 0.3941 - learning_rate: 0.0100
Epoch 2/5


/opt/anaconda3/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


345/345 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.0938 - loss: 0.3287 - val_accuracy: 0.1664 - val_loss: 0.3948 - learning_rate: 0.0100
Epoch 3/5
345/345 ━━━━━━━━━━━━━━━━━━━━ 77s 223ms/step - accuracy: 0.1373 - loss: 0.3308 - val_accuracy: 0.1823 - val_loss: 0.3906 - learning_rate: 0.0100
Epoch 4/5
345/345 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.2812 - loss: 0.3172 - val_accuracy: 0.1845 - val_loss: 0.3903 - learning_rate: 0.0100
Epoch 5/5
345/345 ━━━━━━━━━━━━━━━━━━━━ 83s 240ms/step - accuracy: 0.1586 - loss: 0.3265 - val_accuracy: 0.2106 - val_loss: 0.3911 - learning_rate: 0.0100


In [163]:
characters

['homer_simpson',
 'ned_flanders',
 'moe_szyslak',
 'lisa_simpson',
 'bart_simpson',
 'marge_simpson',
 'krusty_the_clown',
 'principal_skinner',
 'charles_montgomery_burns',
 'milhouse_van_houten']

In [167]:
test_path =r'/Users/hetan2/Desktop/simpsons /archive/simpsons_dataset/lisa_simpson/pic_0897.jpg'
img = cv.imread(test_path)
def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img

In [168]:
predict = model.predict(prepare(img))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [169]:
print(characters[np.argmax(predict[0])])

lisa_simpson
